In [1]:
import os
import time
import requests
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.colors as mcolors
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# 현재 작업 디렉토리 경로 얻기
current_directory = os.getcwd()
# 나눔바른고딕 글꼴 파일 경로 지정
font_path = os.path.join(current_directory, "NanumBarunGothic.ttf")

# 글꼴이 있는지 확인
if os.path.exists(font_path):
    # 폰트 추가
    fm.fontManager.addfont(font_path)

    # 캐시를 새로 고치지 않고 직접 폰트 설정
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc("font", family=font_prop.get_name())
    plt.rcParams["axes.unicode_minus"] = False  # 마이너스 부호 출력 설정

In [3]:
# df = pd.read_csv('Interpark_books_dataset.csv')
# df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'Interpark_books_dataset.csv'

In [ ]:
# google.upload
df = pd.read_csv('Interpark_books_dataset.csv')
df.info()

In [ ]:
df.head()

In [ ]:
df['Sales'].describe()

In [ ]:
# Type별 데이터 수를 계산합니다.
type_counts = df['Category'].value_counts()

# Type별 데이터 수를 데이터 프레임으로 변환합니다.
type_counts_df = type_counts.reset_index()
type_counts_df.columns = ['Category', 'Sales']

# 시각화
plt.figure(figsize=(14, 10))
sns.boxplot(y='Sales', data=type_counts_df, color='magenta')
plt.title('Boxplot of Category Counts')
plt.xlabel('Category Sales')
plt.show()

In [ ]:
df.Category.value_counts()

저희의 특정 칼럼들을 지울 필요가 있다고 생각했습니다.

- 목적과 관련성: 초중고생 학습서, 자격서/수험서, 전공도서/대학교재와 같은 책들은 주로 교육적인 목적이나 전문 지식 습득을 목적으로 합니다. 이들은 독서량 증진을 위한 일반적인 독서 활동과는 목적이 다를 수 있습니다.

- 대상 독자의 특성: 이러한 유형의 책들은 주로 특정 시험이나 학습 목표를 가진 독자들에 의해 구매됩니다. 따라서 이들 데이터는 일반 독서 패턴을 분석하는 데 있어서 다소 왜곡 요소가 될 수 있습니다.

-  분석의 목적 설정: 독서량 증진을 목적으로 하는 경우, 일반 독서 활동에 집중된 데이터가 필요합니다. 이를 통해 독서 문화의 변화와 관련된 인사이트를 정확하게 도출할 수 있습니다.

위와 같은 이유로 초중고생 학습서, 자격서/수험서, 전공도서/대학교재와 같은 데이터를 분석에서 제외하여 일반 독서 활동과 관련된 인사이트를 더욱 정확하게 얻고자 합니다.

In [ ]:
filtered_df = df[~df['Category'].isin(['초중고생학습서', '국어/외국어/사전', '자격서/수험서', '전공도서/대학교재', '잡지'])]

# 필터링된 데이터셋 확인
print(filtered_df['Category'].value_counts())

In [ ]:
# 여행, 취미/레저를 예술/대중문화에 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['여행', '취미/레저', '예술/대중문화'], '예술/대중문화/취미')

# 유아동전집 및 유아, 아동을 유아동으로 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['유아동전집', '유아', '아동'], '유아동')

# 일본도서를 외국도서로 합치기
filtered_df['Category'] = filtered_df['Category'].replace('일본도서', '외국도서')

# 역사와 사회과학을 사회과학 / 역사로 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['역사', '사회과학'], '사회과학 / 역사')

# 건강/미용을 가정과 생활로 합치기
filtered_df['Category'] = filtered_df['Category'].replace('건강/미용', '가정과 생활')


# 합쳐진 결과 확인
print(filtered_df['Category'].value_counts())

In [ ]:
filtered_df.info()

In [ ]:
filtered_df

In [ ]:
# 연령대에 따른 도서 선호 시각화
def visualize_preference_by_age(data):
    # 연령대 그룹 정의
    age_groups = {
        1: '10대',
        2: '20대',
        3: '30대',
        4: '40대',
        5: '50대',
        6: '60세 이상'
    }

    # 연령대에 따라 데이터 그룹화
    age_categories = data.groupby('Age')['Category'].value_counts()

    # 시각화
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    for i, (age_code, age_label) in enumerate(age_groups.items()):
        row = i // 3
        col = i % 3

        # 해당 연령대의 카테고리 파이차트
        if age_code in age_categories.index.get_level_values(0):
            preferences = age_categories[age_code]
            axes[row, col].pie(preferences, labels=preferences.index, autopct='%1.1f%%', startangle=90)
            axes[row, col].set_title(f'{age_label} Preferences')

    plt.tight_layout()
    plt.show()

# 필터된 데이터를 사용하여 시각화 함수 호출
visualize_preference_by_age(filtered_df)

In [ ]:
filtered_df.info()

In [ ]:
filtered_df.head()

In [ ]:
filtered_df['Category'].value_counts()

In [ ]:
######################################################################
# 카테고리별로 색상을 지정했는데 연하게 보여서 색상을을 임의로 지정했습니다 #
######################################################################
def visualize_preference_by_age(data):
    age_groups = {
        1: '10대', 2: '20대', 3: '30대',
        4: '40대', 5: '50대', 6: '60세 이상'
    }
    #8c564b #7f7f7f #848484
    # 진한 색상의 커스텀 리스트
    custom_colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
        '#D500B7', '#bcbd22', '#17becf', '#393b79', '#637939', '#8c6d31', '#843c39',
        '#7b4173', '#5254a3', '#6b6ecf', '#9c9ede', '#cedb9c', '#e7ba52'
    ]

    # 모든 카테고리 추출 및 색상 맵 생성
    all_categories = data['Category'].unique()
    color_map = dict(zip(all_categories, custom_colors[:len(all_categories)]))
    color_map['그 외 장르'] = '#808080'  # 그 외 장르는 회색으로 설정

    fig, axes = plt.subplots(2, 3, figsize=(20, 14))

    for i, (age_code, age_label) in enumerate(age_groups.items()):
        row, col = divmod(i, 3)

        # 해당 연령대의 카테고리 선호도 계산
        preferences = data[data['Age'] == age_code]['Category'].value_counts()

        # 상위 8개 카테고리 선택 및 나머지를 '그 외 장르'로 묶기
        top_8 = preferences.nlargest(8)
        others = pd.Series({'그 외 장르': preferences.sum() - top_8.sum()})
        plot_data = pd.concat([top_8, others])

        # 색상 리스트 생성
        colors = [color_map[cat] for cat in plot_data.index]

        # 파이 차트 그리기
        wedges, texts, autotexts = axes[row, col].pie(plot_data, labels=plot_data.index, autopct='%1.1f%%',
                                                      startangle=90, colors=colors)

        # 레이블 텍스트 크기 조정
        plt.setp(texts, size=8)
        plt.setp(autotexts, size=8, weight="bold")

        axes[row, col].set_title(f'{age_label} Preferences', size=14)

    plt.tight_layout()
    plt.show()

# 함수 호출
visualize_preference_by_age(filtered_df)

연령에 따른 관심사 변화:

- 젊은 층(10-20대)의 특징은 다른그룹에는 보이지 않는 만화/라이트노벨의 독서가 선호되는 점입니다.
- 10대가 지나서 부터는 자기계발서, 인문 경제경영분야의 비중이 증가합니다.
- 30-40대에서는 유아동 분야의 도서구매율이 늘어납니다. 이는 결혼을 하고 아이를 위한 도서구매가 늘어남을 알 수 있습니다.
- 40대 이상에서 부터 사회과학, 역사분야의 비중이 눈에 띄게 높아집니다.
- 50대와 60대 이상에서는 가정과 생활 관련 서적의 선호도가 증가합니다.


일관된 인기 장르:

- 소설과 시/에세이, 인문분야는 전 연령대에서 상위권을 차지하는 보편적 인기 장르입니다.


마케팅 및 출판 전략에의 시사점:

- 세대를 아우르는 소설과 시/에세이, 인문분야는 안정적인 출판 분야입니다.
- 젊은층을 위한 디지털 콘텐츠(예: 웹소설, 웹툰)와 장년층을 위한 전통적 형태의 출판물 균형이 중요합니다.